### Import and Load model

In [2]:
import time
import sys
sys.path.append("../")
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np 
from scipy.stats import entropy
from tqdm import tqdm

import torch
from torch import nn
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from bayesian_torch.models.dnn_to_bnn import dnn_to_bnn

from utils.plots import plot_pr,plot_roc, plot_dist, plot_violin
from utils.metircs import accuracy,mutual_info,nll, ece,predictive_entropy
from utils.visual import ProgressMeter, AverageMeter, Summary
from data_utils.get_datasets import get_dataset
from model_utils.get_models import get_model
from uncertainty.mc_dropout import mc_dropout_predict
from uncertainty.test_time_aug import test_time_aug_predict
from uncertainty.deep_ensemble import deep_ensembel_predict,EnsembleModel
from uncertainty.baseline_softmax import baseline_softmax_predict
from uncertainty.bnn_svi import bnn_svi_predict
from adversarial.adversarial_attack import fgsm_attack, pgd_attack

In [3]:
device = torch.device('cuda:0')
model = get_model("vgg16", 10, use_torchvision=False)
model = model.to(device)
model.eval()

checkpoint = torch.load("../saved_models/deterministic/vgg16/2023_11_24_15_25_21/vgg16_best_model_93.62.pth",map_location=device)
model.load_state_dict(checkpoint['state_dict'])

use private models...


<All keys matched successfully>

In [4]:
summary(model,(3,32,32))
print(model)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,928
       BatchNorm2d-5           [-1, 64, 32, 32]             128
              ReLU-6           [-1, 64, 32, 32]               0
         MaxPool2d-7           [-1, 64, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]          73,856
       BatchNorm2d-9          [-1, 128, 16, 16]             256
             ReLU-10          [-1, 128, 16, 16]               0
           Conv2d-11          [-1, 128, 16, 16]         147,584
      BatchNorm2d-12          [-1, 128, 16, 16]             256
             ReLU-13          [-1, 128, 16, 16]               0
        MaxPool2d-14            [-1, 12

In [5]:
#random Gaussian noise fed into an image classifier gives a “prediction confidence” or predicted class probability of 91%
noise = torch.randn(64,3,32,32).to(device)
prob = torch.softmax(model(noise), axis=1)
print(torch.max(prob,axis=1))

torch.return_types.max(
values=tensor([0.5996, 0.5407, 0.5734, 0.3705, 0.4577, 0.6635, 0.3991, 0.6432, 0.5598,
        0.4227, 0.5568, 0.3850, 0.4086, 0.4935, 0.5098, 0.4219, 0.8335, 0.4066,
        0.4242, 0.4654, 0.5902, 0.5511, 0.3518, 0.6379, 0.4552, 0.7202, 0.4472,
        0.3843, 0.4255, 0.5694, 0.6005, 0.4969, 0.5665, 0.3583, 0.5034, 0.6286,
        0.5771, 0.8949, 0.7062, 0.6048, 0.5385, 0.5426, 0.6007, 0.6888, 0.4173,
        0.3707, 0.5325, 0.5984, 0.5838, 0.5409, 0.7255, 0.7124, 0.7267, 0.6357,
        0.6136, 0.6585, 0.7255, 0.4039, 0.5807, 0.4513, 0.6265, 0.7712, 0.4521,
        0.5027], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([4, 4, 4, 6, 4, 4, 6, 4, 4, 4, 4, 4, 4, 6, 4, 2, 4, 2, 4, 4, 4, 4, 2, 4,
        2, 4, 4, 4, 6, 4, 4, 2, 4, 4, 4, 4, 4, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4], device='cuda:0'))


In [6]:
%reload_ext autoreload
%autoreload 2

### MC Dropout

In [7]:
print(model.classifier[2].training)
model.classifier[2].training = True  # 打开test dropout
print(model.classifier[2].training)

False
True


In [ ]:
datanames = ["cifar10","mnist", "fashionmnist","svhn"]
probs_list = []
targets_list = []
for dataname in datanames:
    val_transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(
                (0.4914, 0.4822, 0.4465),
                (0.2023, 0.1994, 0.2010))
    ]
    )
    if dataname=="mnist" or dataname=="fashionmnist":
        val_transform.transforms.insert(2,transforms.Lambda(lambda x:x.repeat(3,1,1)))
    _, val_dataset = get_dataset(dataname,"../data",None,val_transform)
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=128, shuffle=False,
        num_workers=4, pin_memory=True)

    probs, targets = mc_dropout_predict(val_loader, model, device, 20)
    probs_list.append(probs.cpu().detach().numpy())
    targets_list.append(targets.cpu().detach().numpy())

 * Time 0.098 Acc@1 93.570
 * Time 0.107 Acc@1 7.370
 * Time 0.186 Acc@1 10.080


#### misclassified

In [ ]:
print(probs_list[0].shape)

In [ ]:
cifar10_mi =  mutual_info(probs_list[0])*-1
cifar10_probs = np.mean(probs_list[0],axis=1)
cifar10_target = targets_list[0]

In [ ]:
cifar10_pred = np.argmax(cifar10_probs,axis=1)
labels = (cifar10_pred == cifar10_target).astype(int)#正确分类标为1，误分类标为0
random_pred = np.random.rand(labels.shape[0])

In [ ]:
succ_mi_mean = cifar10_mi[labels==1].mean()
err_mi_mean = cifar10_mi[labels==0].mean()
print(f"正确分类样本数{labels.sum()},平均MI:{succ_mi_mean:.3f}\n错误分类样本数{(1-labels).sum()},平均MI:{err_mi_mean:.3f}")

In [ ]:
cifar10_ent = predictive_entropy(probs_list[0])*-1
succ_ent_mean = cifar10_ent[labels==1].mean()
err_ent_mean = cifar10_ent[labels==0].mean()
print(f"正确分类样本数{labels.sum()},平均AU:{succ_ent_mean:.3f}\n错误分类样本数{(1-labels).sum()},平均AU:{err_ent_mean:.3f}")

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(20, 12))
plot_roc(axes[0][0], labels,cifar10_mi,"mi/succ")
plot_pr(axes[0][1],labels,random_pred,"random/succ")
plot_pr(axes[0][2],labels,cifar10_mi,"mi/succ")
plot_roc(axes[1][0], labels,cifar10_ent,"entropy/succ")
plot_pr(axes[1][1],labels,random_pred,"random/succ")
plot_pr(axes[1][2],labels,cifar10_ent,"entropy/succ")
plt.tight_layout()

In [ ]:
labels2 = (cifar10_pred != cifar10_target).astype(int)#误分类标为1，正确分类标为0
cifar10_mi2 = -cifar10_mi
random_pred2 = -random_pred
cifar10_ent2 = -cifar10_ent 

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(20, 12))
plot_roc(axes[0][0], labels2,cifar10_mi2,"mi/err")
plot_pr(axes[0][1],labels2,random_pred2,"random/err")
plot_pr(axes[0][2],labels2,cifar10_mi2,"mi/err")
plot_roc(axes[1][0], labels2,cifar10_ent2,"entropy/err")
plot_pr(axes[1][1],labels2,random_pred2,"random/err")
plot_pr(axes[1][2],labels2,cifar10_ent2,"entropy/err")
plt.tight_layout()

#### OOD

In [ ]:
mi_list = []
for i, probs in enumerate(probs_list):
    mi = mutual_info(probs)
    mi_list.append(mi)

In [ ]:
ent_list = []
for i, probs in enumerate(probs_list):
    ent = predictive_entropy(probs)
    ent_list.append(ent)

In [ ]:
#绘制mi,熵的分布
fig, axes = plt.subplots(1,2, figsize=(13, 6))
plot_dist(axes[0],mi_list,datanames,"mi")
plot_dist(axes[1], ent_list,datanames,"entropy")
plt.tight_layout()

In [ ]:
#绘制小提琴图
fig, axes = plt.subplots(1,4, figsize=(25, 6))
plot_violin(axes,mi_list,datanames,"mi")
fig, axes = plt.subplots(1,4, figsize=(25, 6))
plot_violin(axes, ent_list,datanames,"entropy")
plt.tight_layout()

In [ ]:
#IND样本视为正样本， mnist样本视为负样本
ood_idx=1
print(f"cifar10 test nums:{len(mi_list[0])},mnist test nums:{len(mi_list[1])}")
labels = [1]*len(mi_list[0])+[0]*len(mi_list[ood_idx])
mi = np.concatenate([mi_list[0],mi_list[ood_idx]])*-1
ent = np.concatenate([ent_list[0],ent_list[ood_idx]])*-1
random_pred = np.random.rand(len(labels))*-1

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(20, 12))
plot_roc(axes[0][0], labels,mi,"mi/ind")
plot_pr(axes[0][1],labels,random_pred,"random/ind")
plot_pr(axes[0][2],labels,mi,"mi/ind")
plot_roc(axes[1][0], labels,ent,"entropy/ind")
plot_pr(axes[1][1],labels,random_pred,"random/ind")
plot_pr(axes[1][2],labels,ent,"entropy/ind")
plt.tight_layout()

In [ ]:
#IND样本视为负样本， mnist样本视为正样本
labels2 = [0]*len(mi_list[0])+[1]*len(mi_list[1])
mi2 =  -mi
ent2 = -ent
random_pred2 = -random_pred

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(20, 12))
plot_roc(axes[0][0], labels2,mi2,"mi/ood")
plot_pr(axes[0][1],labels2,random_pred2,"random/ood")
plot_pr(axes[0][2],labels2,mi2,"mi/ood")
plot_roc(axes[1][0], labels2,ent2,"entropy/ood")
plot_pr(axes[1][1],labels2,random_pred2,"random/ood")
plot_pr(axes[1][2],labels2,ent2,"entropy/ood")
plt.tight_layout()

#### advasarial attack

In [ ]:
_, val_dataset = get_dataset("cifar10","../data",None,val_transform)
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=1, shuffle=False,
    num_workers=4, pin_memory=True)

In [ ]:
adv_probs_list = []
origin_probs_list = []
method = "pgd"
epsilon = 0.01
correct = 0
err = 0
num_correct=0
num_monte_carlo = 20
for images, targets in tqdm(val_loader):
    images, targets = images.to(device), targets.to(device)
    origin_outputs = []
    for _ in range(num_monte_carlo):  # add mc_dropout
        prob = torch.softmax(model(images), axis=1)
        origin_outputs.append(prob)
    origin_outputs = torch.stack(origin_outputs, dim=0)
    origin_prob_mean = torch.mean(origin_outputs, dim=0)
    _, origin_pred = torch.max(origin_prob_mean,axis=1)
    # num_correct += (origin_pred == targets).sum().item()
    if origin_pred.item() != targets.item():
        err += 1
        continue

    if method == "fgsm":
        perturbed_images = fgsm_attack(model, images, targets, epsilon)
    elif method == "pgd":
        perturbed_images = pgd_attack(model, images, targets, epsilon)

    adv_outputs = []
    for _ in range(num_monte_carlo):  # add mc_dropout
        prob = torch.softmax(model(perturbed_images), axis=1)
        adv_outputs.append(prob)
    adv_outputs = torch.stack(adv_outputs, dim=0)
    adv_prob_mean = torch.mean(adv_outputs, dim=0)
    _, adv_pred = adv_prob_mean.max(1, keepdim=True)
    adv_probs_list.append(adv_outputs.cpu().detach().numpy())
    origin_probs_list.append(origin_outputs.cpu().detach().numpy())

    if adv_pred.item() == targets.item():
        correct += 1

adv_acc = correct/float(len(val_loader)-err)
print(
    f"Epsilon: {epsilon}\tTest Accuracy = {correct} / {len(val_loader)} = {adv_acc}\tIgnore err = {err}")

In [ ]:
adv_probs = np.concatenate(adv_probs_list,axis=1).transpose(1,0,2)
origin_probs = np.concatenate(origin_probs_list,axis=1).transpose(1,0,2)
print(adv_probs.shape,origin_probs.shape)

In [ ]:
mi_list = []
for i, probs in enumerate([origin_probs,adv_probs]):
    mi_list.append(mutual_info(probs))

In [ ]:
print(mi_list[0])
print(mi_list[1])

In [ ]:
ent_list = []
for i, probs in enumerate([origin_probs,adv_probs]):
    ent = predictive_entropy(probs)
    ent_list.append(ent)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(13, 6))
plot_dist(axes[0],mi_list,["original","advasarial"],"mi")
plot_dist(axes[1], ent_list,["original","advasarial"],"entropy")
plt.tight_layout()

In [ ]:
#原始样本视为正样本， 攻击样本视为负样本
labels = [1]*len(mi_list[0])+[0]*len(mi_list[1])
mi = np.concatenate([mi_list[0],mi_list[1]])*-1
ent = np.concatenate([ent_list[0],ent_list[1]])*-1
random_pred = np.random.rand(len(labels))*-1

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(20, 12))
plot_roc(axes[0][0], labels,mi,"mi/origin")
plot_pr(axes[0][1],labels,random_pred,"random/origin")
plot_pr(axes[0][2],labels,mi,"mi/origin")
plot_roc(axes[1][0], labels,ent,"entropy/origin")
plot_pr(axes[1][1],labels,random_pred,"random/origin")
plot_pr(axes[1][2],labels,ent,"entropy/origin")
plt.tight_layout()

In [ ]:
#原始样本视为正样本， 攻击样本视为负样本
labels2 = [0]*len(mi_list[0])+[1]*len(mi_list[1])
mi2 =  -mi
ent2 = -ent
random_pred2 = -random_pred

In [ ]:
fig, axes = plt.subplots(2,3, figsize=(20, 12))
plot_roc(axes[0][0], labels2,mi2,"mi/adv")
plot_pr(axes[0][1],labels2,random_pred2,"random/adv")
plot_pr(axes[0][2],labels2,mi2,"mi/adv")
plot_roc(axes[1][0], labels2,ent2,"entropy/adv")
plot_pr(axes[1][1],labels2,random_pred2,"random/adv")
plot_pr(axes[1][2],labels2,ent2,"entropy/adv")
plt.tight_layout()